<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Flux-on-object" data-toc-modified-id="Flux-on-object-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Flux on object</a></span><ul class="toc-item"><li><span><a href="#Side-by-side-comparison" data-toc-modified-id="Side-by-side-comparison-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Side by side comparison</a></span></li></ul></li><li><span><a href="#Signal" data-toc-modified-id="Signal-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Signal</a></span><ul class="toc-item"><li><span><a href="#IR-Pulse" data-toc-modified-id="IR-Pulse-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>IR Pulse</a></span></li><li><span><a href="#Solar" data-toc-modified-id="Solar-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Solar</a></span></li></ul></li><li><span><a href="#Noise" data-toc-modified-id="Noise-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Noise</a></span><ul class="toc-item"><li><span><a href="#Noise-sources" data-toc-modified-id="Noise-sources-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Noise sources</a></span></li><li><span><a href="#Noise-comparison" data-toc-modified-id="Noise-comparison-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Noise comparison</a></span></li></ul></li><li><span><a href="#SNR" data-toc-modified-id="SNR-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>SNR</a></span></li><li><span><a href="#All-component" data-toc-modified-id="All-component-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>All component</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import os
import sys
from IPython.display import display, HTML
from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go
init_notebook_mode(connected=True)  # for Jupyter Lab notebook

CURRENT_DIR = os.path.dirname(os.getcwd())
sys.path.append(os.path.dirname(CURRENT_DIR+'/func'))
from func.photonic_func import Photonic

In [2]:
def overide_phototonic(photonic):
    photonic.sensor_.loc['ToF_850','DkSig_e_s'] = 50000 # [e-/sec]
    photonic.sensor_.loc['ToF_850', 'epi_thick_um'] = 9.5 # [um]
    photonic.update_photonic()
    return photonic

photonic = Photonic()
display(HTML(photonic.config_.to_html()))
display(HTML(photonic.sensor_.to_html()))
display(HTML(photonic.light_.to_html()))

,Light,Scene,Lens,Sensor,Op
Name,,,,,
Cfg1,Laser1,Room_3,Lens1,Sensor1_850,CW_30fps
Cfg2,VCSEL2,Room_4,Lens2,Sensor1_550,CW_60fps
Cfg3,Laser1,Road_night_200,Lens1,Sensor1_850,pls_30_1000
fake_tof_night_850,fake_tof_850,Room_5,fake_tof_lens,ToF_850,fake_tof
fake_tof_night_940,fake_tof_940,Room_5,fake_tof_lens,ToF_940,fake_tof
fake_tof_night_1375,fake_tof_1375,Room_5,fake_tof_lens,ToF_1375,fake_tof
fake_tof_night_1550,fake_tof_1550,Room_5,fake_tof_lens,ToF_1550,fake_tof
fake_tof_day_850,fake_tof_850,Outdoor_5,fake_tof_lens,ToF_850,fake_tof
fake_tof_day_940,fake_tof_940,Outdoor_5,fake_tof_lens,ToF_940,fake_tof


,PixSz_um,Wavelength,Semiconductor,epi_thick_um,QE,FF,Res_H,Res_V,RdNoise_e,DkSig_e_s,Cfd_fF,CDS
Name,,,,,,,,,,,,
Sensor1_850,3.0,850,Si,10,NaN,0.7,640,480,5,5000,NaN,True
Sensor1_550,3.0,550,Si,10,NaN,0.7,640,480,5,5000,NaN,True
ToF_850,7.5,850,Si,10,NaN,1.0,640,480,5,5000,10.0,False
ToF_940,7.5,940,Si,10,NaN,1.0,640,480,5,5000,10.0,False
ToF_1375,7.5,1375,Ge,1,NaN,1.0,640,480,5,5000,10.0,False
ToF_1550,7.5,1550,Ge,1,NaN,1.0,640,480,5,5000,10.0,False


,PeakPower_W,WaveLength_nm,Transmission,Hfov_deg,Vfov_deg,Number_units,Voltage_V,Current_A,Link
Name,,,,,,,,,
Laser1,20.0,850,0.95,60,60,1,NaN,NaN,NaN
VCSEL2,10.0,940,0.95,60,60,1,NaN,NaN,NaN
L1550G1,2.1,1550,0.95,40,20,4,1.9,6.000000,https://www.thorlabs.com/drawings/471c86010a0aa6dc-F53DE964-0AE3-E9AF-406B86F1AA7C426E/L1550G1-SpecSheet.pdf
MCM-105,20.0,1550,0.95,40,20,1,10.7,9.090909,https://www.laserdiodesource.com/files/pdfs/laserdiodesource_com/product-2924/1550nm_20W_Fiber_Coupled_Module_Seminex-1459186218.pdf
VCSEL_940_ii_vi,4.0,940,0.95,40,20,1,2.0,6.000000,https://www.ii-vi.com/wp-content/uploads/2019/07/940nm-Multi-Mode-High-Power-VCSEL-Array-web-2-1.pdf
MCM-124,28.0,1375,0.95,40,20,1,11.5,12.500000,https://www.laserdiodesource.com/files/pdfs/laserdiodesource_com/product-2924/1550nm_20W_Fiber_Coupled_Module_Seminex-1459186218.pdf
fake_tof_850,25.0,850,0.95,70,60,2,NaN,NaN,NaN
fake_tof_940,19.0,940,0.95,70,60,2,NaN,NaN,NaN
fake_tof_1375,11.0,1375,0.95,70,60,2,NaN,NaN,NaN


# Flux on object

## Side by side comparison

In [3]:
# test_cfg = ['fake_tof_night_850','fake_tof_day_850',
#             'fake_tof_night_940','fake_tof_day_940',
#             'fake_tof_night_1375','fake_tof_day_1375',
#             'fake_tof_night_1550','fake_tof_day_1550']
# data = [0,1,2,3,4,5,6,7, 8, 9, 10, 11, 12, 13, 14, 15]
test_cfg = ['fake_tof_day_940','fake_tof_day_1375']
data = [0,1,2,3]
for i, cfg in enumerate(test_cfg):
    photonic = Photonic(config=cfg)
    photonic = overide_phototonic(photonic)
    dist_vec = np.array([0.5,1,2,5,10])
    trace0 = go.Scatter(x=dist_vec,
                        y=1000 * photonic.wallFlux(dist_vec=dist_vec),   # Select 'lines', 'markers' or 'lines+markers'
                        mode='lines+markers', name=cfg[9:]+' IR')
    trace1 = go.Scatter(x=dist_vec,
                        y=1000 * photonic.wallFlux(dist_vec=dist_vec, light_type='solar'), mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                        name=(cfg[9:]+' Solar'))
    data[2*i] = trace0
    data[2*i+1] = trace1

layout = dict(title='Photonic simulation - Flux on wall - Fake ToF - Constrained to Same Illumination Power',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='Flux [mW/m**2]',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))

# Signal

## IR Pulse

In [4]:
test_cfg = ['fake_tof_night_850','fake_tof_day_850',
            'fake_tof_night_940','fake_tof_day_940',
            'fake_tof_night_1375','fake_tof_day_1375',
            'fake_tof_night_1550','fake_tof_day_1550']
data = [0,1,2,3,4,5,6,7]
data2 = [0,1,2,3,4,5,6,7]
# test_cfg = ['fake_tof_day_940','fake_tof_day_1375']
# data = [0,1]
for i, cfg in enumerate(test_cfg):
    photonic = Photonic(config=cfg)
    photonic = overide_phototonic(photonic)
    dist_vec = np.array([0.5,0.7,1,1.5,2,3,4,5,7,10])
    signal = photonic.photoelectron2(dist_vec=dist_vec)
    trace0 = go.Scatter(x=dist_vec, y=signal,   # Select 'lines', 'markers' or 'lines+markers'
                        mode='lines+markers', name=cfg[9:])
    data[i] = trace0

layout = dict(title='Photonic simulation - Signal',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='Noise [e-]',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))

## Solar

In [5]:
test_cfg = ['fake_tof_day_850','fake_tof_day_940','fake_tof_day_1375','fake_tof_day_1550']
data = [0,1,2,3]
# test_cfg = ['fake_tof_day_940','fake_tof_day_1375']
# data = [0,1]
for i, cfg in enumerate(test_cfg):
    photonic = Photonic(config=cfg)
    photonic = overide_phototonic(photonic)
    dist_vec = np.array([0.5,0.7,1,1.5,2,3,4,5,7,10])
    solar=photonic.photoelectron(
        siliconFlux=photonic.siliconFlux(
        wall_flux=photonic.wallFlux(dist_vec=dist_vec,light_type='solar')))
    trace0 = go.Scatter(x=dist_vec, y=solar,   # Select 'lines', 'markers' or 'lines+markers'
                        mode='lines+markers', name=cfg[9:])
    data[i] = trace0

layout = dict(title='Photonic simulation - Solar Flux',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='Noise [e-]',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))

In [6]:
photonic = Photonic()
solar_W_m2_um, wavelength_um = photonic.solarSpectrum_W_m2_um()

data = go.Scatter(x=wavelength_um, y=solar_W_m2_um,   # Select 'lines', 'markers' or 'lines+markers'
                    mode='lines+markers')

layout = dict(title='Solar Spectrum',
              xaxis=dict(title='Wavelength [µm]', type='linear'),  # Select 'log' or 'linear'
              yaxis=dict(title='Noise [e-]',type='linear'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))

# Noise

## Noise sources

In [7]:
# test_cfg = ['fake_tof_night_850','fake_tof_day_850',
#             'fake_tof_night_940','fake_tof_day_940',
#             'fake_tof_night_1375','fake_tof_day_1375',
#             'fake_tof_night_1550','fake_tof_day_1550']
# data = [0,1,2,3,4,5,6,7]
cfg = 'fake_tof_day_1375'
# data = [0]
# for i, cfg in enumerate(test_cfg):
photonic = Photonic(config=cfg)
photonic = overide_phototonic(photonic)
dist_vec = np.array([0.5,0.7,1,1.5,2,3,4,5,7,10])
signal, noise, SNR = photonic.signal_to_noise_ratio(dist_vec=dist_vec)
sz = len(noise['noise_photon_shot'])

trace0 = go.Scatter(x=dist_vec, y=noise['noise_photon_shot'],   # Select 'lines', 'markers' or 'lines+markers'
                    mode='lines+markers', name='photon shot '+cfg[9:])
trace1 = go.Scatter(x=dist_vec, y=sz*[noise['dark_noise']],   # Select 'lines', 'markers' or 'lines+markers'
                    mode='lines+markers', name='dark '+cfg[9:])
trace2 = go.Scatter(x=dist_vec, y=sz*[noise['kTC_noise']],   # Select 'lines', 'markers' or 'lines+markers'
                    mode='lines+markers', name='kTC '+cfg[9:])
data = [trace0, trace1, trace2]

layout = dict(title='Photonic simulation - Major Noise Sources',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='Noise [e-]',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))

## Noise comparison

In [8]:
test_cfg = ['fake_tof_night_850','fake_tof_day_850',
            'fake_tof_night_940','fake_tof_day_940',
            'fake_tof_night_1375','fake_tof_day_1375',
            'fake_tof_night_1550','fake_tof_day_1550']
data = [0,1,2,3,4,5,6,7]
data2 = [0,1,2,3,4,5,6,7]
# test_cfg = ['fake_tof_day_940','fake_tof_day_1375']
# data = [0,1]
for i, cfg in enumerate(test_cfg):
    photonic = Photonic(config=cfg)
    photonic = overide_phototonic(photonic)
    dist_vec = np.array([0.5,0.7,1,1.5,2,3,4,5,7,10])
    signal, noise, SNR = photonic.signal_to_noise_ratio(dist_vec=dist_vec)
    trace0 = go.Scatter(x=dist_vec, y=sum(noise.values()),   # Select 'lines', 'markers' or 'lines+markers'
                        mode='lines+markers', name=cfg[9:])
    data[i] = trace0
    trace1 = go.Scatter(x=dist_vec, y=SNR,   # Select 'lines', 'markers' or 'lines+markers'
                        mode='lines+markers', name=cfg[9:])
    data2[i] = trace1
    trace20 = go.Scatter(x=dist_vec, y=10*[5],   # Select 'lines', 'markers' or 'lines+markers'
                        mode='lines', name='Low SNR limit for ToF')
data2.append(trace20)
layout = dict(title='Photonic simulation - Summed Noise',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='Noise [e-]',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))

# SNR

In [9]:
layout = dict(title='Photonic simulation - SNR',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='SNR',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data2, layout=layout))

# All component

In [10]:
photonic = Photonic(config='fake_tof_day_1375')
photonic = overide_phototonic(photonic)
dist_vec = np.array([0.5,0.7,1,1.5,2,3,4,5,7,10])
trace0 = go.Scatter(x=dist_vec,
                    y=1000 * photonic.wallFlux(dist_vec=dist_vec), mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                    name='wallFlux [W/m^2]')
trace01 = go.Scatter(x=dist_vec,
                    y=1000 * photonic.wallFlux(dist_vec=dist_vec, light_type='solar'), mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                    name='Solar wallFlux [W/m^2]')
trace1 = go.Scatter(x=dist_vec,
                    y=1000 * photonic.siliconFlux2(dist_vec=dist_vec), mode='lines+markers',
                    name='siliconFlux [W/m^2]')
trace2 = go.Scatter(x=dist_vec,
                    y=photonic.photoelectron2(dist_vec=dist_vec), mode='lines+markers',
                    name='photoelectrons / frame')
signal, noise, SNR = photonic.signal_to_noise_ratio(dist_vec=dist_vec)
trace3 = go.Scatter(x=dist_vec,
                    y=SNR, mode='lines+markers',
                    name='SNR')
trace4 = go.Scatter(x=[0.5],
                    y=[0.2], mode='text', textposition='top right',
                    name='text', text=['Photonic simulation of light created by a light source attached to a camera<br>'
                    +'1. Flux is calculated on a wall at a certain distance assuming CW lighting mode<br>'
                    +'2. Flux on the focal plane of the silicon sensor as imaged from the wall thru the lens<br>'
                    +'3. Photoelectrons per a single burst collect in the photodiode of the pixel'])

data = [trace0, trace01, trace1, trace2, trace3, trace4]

layout = dict(title='Photonic simulation - Flux on wall/sensor, PE count, SNR (fake_tof_day_1375)' ,
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))